In [ ]:
!pip install paho-mqtt

In [ ]:
import paho.mqtt.client as mqtt
import ssl
import time
import sys

# ================= CONFIGURACIÓN =================
# Tus credenciales exactas del ESP32
MQTT_BROKER = "12f114da5d6a4b0d93d624aa7d4982ee.s1.eu.hivemq.cloud"
MQTT_PORT = 8883
MQTT_USER = "esp32"
MQTT_PASS = "Giorpa469"
MQTT_TOPIC_CMD = "esp32/comandos"

# ================= FUNCIONES =================

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("\n✅ CONEXIÓN EXITOSA CON HIVEMQ")
        print("------------------------------------------------")
        print(f"📡 Enviando comandos al tema: {MQTT_TOPIC_CMD}")
        print("💡 Comandos disponibles: LED_ON, LED_OFF")
        print("🚪 Escribe 'SALIR' para terminar")
        print("------------------------------------------------")
    else:
        print(f"\n❌ Error de conexión. Código: {rc}")

def on_publish(client, userdata, mid):
    # Callback opcional para confirmar que el mensaje salió de la PC
    pass

# ================= MAIN =================

# 1. Crear el cliente (Debe tener un ID único diferente al del ESP32)
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1, "PC_Controlador_Terminal")

# 2. Configurar seguridad (OBLIGATORIO para HiveMQ Cloud)
client.tls_set(cert_reqs=ssl.CERT_NONE)
client.username_pw_set(MQTT_USER, MQTT_PASS)

# 3. Asignar callbacks
client.on_connect = on_connect
client.on_publish = on_publish

# 4. Conectar
print("⏳ Conectando al servidor MQTT...")
try:
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
except Exception as e:
    print(f"❌ No se pudo conectar: {e}")
    sys.exit(1)

# 5. Iniciar el loop en segundo plano (hilo separado)
# Esto permite que la conexión se mantenga viva mientras tú escribes
client.loop_start()

# 6. Bucle infinito de escritura
try:
    while True:
        # Esperar input del usuario
        comando = input("Escribe comando >> ").strip().upper()

        if comando == "SALIR":
            break

        if comando: # Si no está vacío
            info = client.publish(MQTT_TOPIC_CMD, comando)
            info.wait_for_publish() # Esperar confirmación de envío
            print(f"   🚀 Enviado: '{comando}'")

except KeyboardInterrupt:
    print("\nInterrupción detectada...")

# 7. Cerrar limpiamente
print("Desconectando...")
client.loop_stop()
client.disconnect()
print("👋 Fin del programa.")

/tmp/ipython-input-2696147511.py:34: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1, "PC_Controlador_Terminal")


⏳ Conectando al servidor MQTT...

✅ CONEXIÓN EXITOSA CON HIVEMQ
------------------------------------------------
📡 Enviando comandos al tema: esp32/comandos
💡 Comandos disponibles: LED_ON, LED_OFF
🚪 Escribe 'SALIR' para terminar
------------------------------------------------
   🚀 Enviado: 'LED_OFF'
   🚀 Enviado: 'LED_ON'
   🚀 Enviado: 'LED_OFF'
   🚀 Enviado: 'LED_ON'
   🚀 Enviado: 'LED_OFF'


In [ ]:
pip install paho-mqtt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 3.5 MB/s eta 0:00:00


In [ ]:
import paho.mqtt.client as mqtt
import ssl
import time
import csv
import json
import sys

# ================= CONFIGURACIÓN MQTT =================
MQTT_BROKER = "12f114da5d6a4b0d93d624aa7d4982ee.s1.eu.hivemq.cloud"
MQTT_PORT = 8883
MQTT_USER = "esp32"
MQTT_PASS = "Giorpa469"
MQTT_TOPIC_DATA = "esp32/mpu6050"  # Tema desde donde ESP32 envía los paquetes JSON

# Donde almacenaremos TODOS los datos recibidos
datos_recibidos = []

# ================= CALLBACKS =================

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("✅ Conexión establecida con HiveMQ")
        client.subscribe(MQTT_TOPIC_DATA)
        print(f"📡 Suscrito a: {MQTT_TOPIC_DATA}")
    else:
        print(f"❌ Error de conexión: {rc}")

def on_message(client, userdata, msg):
    global datos_recibidos
    payload = msg.payload.decode("utf-8")

    try:
        paquete = json.loads(payload)

        # Validar estructura
        if "data" in paquete and isinstance(paquete["data"], list):
            print(f"📦 Paquete recibido: {len(paquete['data'])} muestras")

            # Guardar cada lectura individual
            for medicion in paquete["data"]:
                datos_recibidos.append(medicion)

        else:
            print("⚠ Paquete ignorado: formato incorrecto")

    except json.JSONDecodeError:
        print("❌ Error: mensaje no es JSON válido")


# ================= MAIN =================

client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1, "PC_Receptor_JSON")
client.tls_set(cert_reqs=ssl.CERT_NONE)
client.username_pw_set(MQTT_USER, MQTT_PASS)

client.on_connect = on_connect
client.on_message = on_message

print("⏳ Conectando al broker MQTT...")
try:
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
except Exception as e:
    print(f"❌ No se pudo conectar: {e}")
    sys.exit(1)

client.loop_start()

print("\n📌 Recibiendo paquetes JSON... Escribe STOP para guardar en CSV.\n")

try:
    while True:
        comando = input("").strip().upper()
        if comando == "STOP":
            break
except KeyboardInterrupt:
    print("\n⚠ Interrupción detectada...")

# ================= GUARDAR CSV =================
nombre_archivo = f"datos_mqtt_{time.strftime('%Y%m%d_%H%M%S')}.csv"

if datos_recibidos:
    with open(nombre_archivo, "w", newline="", encoding="utf-8") as file:
        columnas = ["fecha","ax","ay","az","gx","gy","gz","a_total","temp"]
        writer = csv.DictWriter(file, fieldnames=columnas)
        writer.writeheader()
        writer.writerows(datos_recibidos)

    print(f"\n💾 CSV guardado con {len(datos_recibidos)} muestras:")
    print(f"📁 Archivo: {nombre_archivo}")
else:
    print("\n⚠ No se recibieron datos. Nada que guardar.")

client.loop_stop()
client.disconnect()
print("👋 Fin del programa.")


/tmp/ipython-input-3037007826.py:52: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1, "PC_Receptor_JSON")


⏳ Conectando al broker MQTT...

📌 Recibiendo paquetes JSON... Escribe STOP para guardar en CSV.

✅ Conexión establecida con HiveMQ
📡 Suscrito a: esp32/mpu6050
📦 Paquete recibido: 50 muestras
📦 Paquete recibido: 50 muestras
📦 Paquete recibido: 50 muestras
📦 Paquete recibido: 50 muestras
STOP

💾 CSV guardado con 200 muestras:
📁 Archivo: datos_mqtt_20251229_163312.csv
👋 Fin del programa.
